# EDS 220 Fall 2022
# Data Regridding

Let's get some practice working with interpolation and data regridding! We'll be working with the same SST datasets used in other notebooks, just to make things easier.

__NOTE: the selection of the correct Python kernel is important!__

This notebook requires the use of Cartopy later on - on Taylor, the "carto" kernel should allow access to this and all other necessary packages.

Import packages first:

In [4]:
# Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import scipy
from scipy import interpolate
import cartopy
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import cartopy.feature as cfeature

### 1) Read in data

To demonstrate the process of converting two-dimensional data to a common grid, we'll use two gridded climate data products:

* The [NOAA Extended Reconstructed Sea Surface Temperature version 5](https://psl.noaa.gov/data/gridded/data.noaa.ersst.v5.html) (ERSSTv5)
* The [Global Precipitation Climatology Product](https://psl.noaa.gov/data/gridded/data.gpcp.html) (GPCP)

Both of these data sets have relatively low spatial resolution: roughly 2 degrees on a side in lat and lon. This makes them quick to load over OpeNDAP. But the locations of the data points are not the same, meaning that to directly compare we need to do our regridding exercise!

First, load the data using the remote OpeNDAP URLs:

In [1]:
# location of ERSSTv5 data
ersst_url="https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc"

# open dataset using remote URL


# display dataset to see what it looks like


In [2]:
# Location of precipitation dataset
gpcp_url="https://psl.noaa.gov/thredds/dodsC/Datasets/gpcp/precip.mon.mean.nc"

# open dataset with remote URL


# display dataset to see what it looks like


To get a visual sense of how the data products are laid out spatially, we can create plots using Matplotlib's `pcolor`. If we wanted to create a production-quality plot, using a map projection with Cartopy would be necessary, but here we're just looking for a quick overview:

In [5]:
# Take time average of ERSST data


# Create a blank figure
plt.figure(figsize=(14,10))
# Plot ERSST time average


<Figure size 1400x1000 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>

In [6]:
# Take time average of GPCP data


# Create a blank figure
plt.figure(figsize=(14,10))
# Plot GPCP time average


<Figure size 1400x1000 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>

### 2) Interpolate to a common grid: xarray built-in functionality

It should be clear that there are differences between these datasets: they're entirely different variables, with different spatial coverages, and slightly different resolutions. Now let's put them on a common grid!

The first method we'll use to do this is the built-in `interp` capacity that is held by xarray objects:

[https://docs.xarray.dev/en/stable/generated/xarray.DataArray.interp.html](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.interp.html)

This is a wrapper around the interpolation functions within the `scipy` package.

In [6]:
# Interpolate GPCP to SST grid


In [7]:
# Display header info for interpolated GPCP


In [8]:
# Create a blank figure
plt.figure(figsize=(14,10))
# Plot interpolated GPCP


<Figure size 1400x1000 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>

### 3) Interpolate to a common grid: Scipy

It's also useful to know how the Scipy package itself handles interpolation, since sometimes data may be read in using formats other than xarray. To do this, we can simply call the Scipy `interpolate` package (imported at the beginning of this notebook) directly.

Here I'll demonstrate an example using the `interp2d` package within interpolate:

[https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html)

In [16]:
# Create interpolation function using Scipy interp2d with input data


Note that the lat and lon values input to `interpolate.interp2d` are the NATIVE (original) lat and lon, NOT the ones we would eventually like to interpolate onto. . That is because the code block above generates an object (`precip_intfn`) which is a Scipy interpolation _function_, which can be evaluated on an arbitrary set of __new__ lon and lat values.

Before performing the interpolation, let's examine the lat and lon coordinates onto which we'd like to perform it:

In [9]:
# Target longitude array


In [10]:
# Target latitude array


These look reasonable, but there is one important thing to notice - the _ordering_ of the latitude points is reversed in the ERSST dataset relative to GPCP. If we do not fix this, it will result in an interpolated precipitation field with an "upside down" appearance (yes, I tried this).

Fortunately, the fix is relatively easy: we just need to create a sorted version of the latitude array before performing the interpolation.

In [11]:
# Sort latitude array


# Display sorted latitudes


In [12]:
# Evaluate interpolation function on new grid: i.e. perform the interpolation


To make it more obvious how the raw vs. interpolated values compare, let's make a real map of both using Cartopy. 

In [13]:
# Create new Plate-Carree projection

# Make blank figure 
fig = plt.figure(figsize=(15, 5))
# Create axes using map projection

# Plot data onto projected axis




<Figure size 1500x500 with 0 Axes>

In [14]:
# Create new Plate-Carree projection

# Make blank figure 
fig = plt.figure(figsize=(15, 5))
# Create axes using map projection

# Plot data onto projected axis




<Figure size 1500x500 with 0 Axes>

Since the input and output grids are not _that_ different from one another, the differences in the interpolated map is somewhat subtle. However, you can see differences in the grid boxes between the upper and lower panels, as well as some shifts in the appearance of the major precipitation features. If your input and output grids differ more from one another, you can expect even larger effects!